In [1]:
import os
from os.path import join
import re
import numpy as np
import shutil
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from glob import glob
from scipy.spatial.transform import Rotation
from skimage.transform import resize
from exr import read_exr

In [2]:
base_dir = '/bigpen/simulator_data/LEC_fast_result'

In [3]:
test_str = "Frame 00001 Rotation: X=-0.0003235038, Y=0.01255963, Z=3.553312E-05, W=0.9999211"
def parse_sim_poses(line: str):
    parsed = line.replace(',','').split(' ')
    poses = [float(parsed[1])] + [float(p[2:]) for p in parsed[3:] if len(p)>2]
    return poses
print(parse_sim_poses(test_str))

[1.0, -0.0003235038, 0.01255963, 3.553312e-05, 0.9999211]


In [4]:
start = 120
skeletons = np.load("/playpen/skeleton-on-surface/figures/LEC_fast/skeletons_3.npy").astype(int)

positions = []
with open('/bigpen/simulator_data/LEC_fast/LEC_Camera Position Data.txt', 'r') as f:
    for line in f:
        if line[0]=='#' or len(line)<2:
            continue
        parsed = line.split(' ')
        positions.append(parse_sim_poses(line))
quaternions = []
with open('/bigpen/simulator_data/LEC_fast/LEC_Camera Quaternion Rotation Data.txt', 'r') as f:
    for line in f:
        if line[0]=='#' or len(line)<2:
            continue
        parsed = line.split(' ')
        quaternions.append(parse_sim_poses(line))

# Convert parsed data to numpy arrays and select only the frames that were skeletonized
pos_arr = np.array(positions)[:,1:]     # remove first column since it's just the frame numbers
print(pos_arr)
quat_arr = np.array(quaternions)[:,1:]
pos_arr = pos_arr[start:start+len(skeletons)]
quat_arr = quat_arr[start:start+len(skeletons)]
rot_mat_arr = Rotation.from_quat(quat_arr).as_matrix() # type: np.ndarray

step = 1
skeletons = skeletons[::step]
pos_arr = pos_arr[::step]
rot_mat_arr = rot_mat_arr[::step]

[[ 0.          0.          0.        ]
 [-0.1219223  -0.06611776  0.391325  ]
 [-0.2345941  -0.1235123   0.7042255 ]
 ...
 [-8.210049    3.513173   16.42544   ]
 [-8.080097    3.530897   16.35147   ]
 [-7.950964    3.544424   16.27737   ]]


In [54]:
# x is down, y is left in camera coords
cx = skeletons.shape[-1]//2
cy = skeletons.shape[-2]//2
fx = 144.2398/320*cx*2
fy = 144.2398/320*cy*2
camera_mat = np.array(
    [[fx, 0, cx],
     [0, fy, cy],
     [0, 0, 1]]
).astype(float)
inv_camera_mat = np.linalg.inv(camera_mat)

In [55]:
# Compute pose matrix
Rt = np.zeros((len(skeletons), 4, 4))
Rt[:,:3,:3] = rot_mat_arr
Rt[:,:3,3] = pos_arr
Rt[:,3,3] = 1

points = []     # points in world coordinates
points_c = []   # points in frames's camera coordinates
frame_inds = []
rng = np.random.default_rng()
for i, sk in enumerate(skeletons):
    sk = np.fliplr(sk)
    # Read in depths
    depths = read_exr(f"/bigpen/simulator_data/LEC_fast/LEC_depth{step*i+120:05}.exr")
    depths = np.fliplr(depths)
    depths = resize(depths, sk.shape)
    coords = np.vstack(np.where(sk>0))
    coords = coords.astype(int)
    zs = depths[coords[0], coords[1]]
    # for n in np.unique(sk)[1:]:
    #     print(n, sk[coords[0], coords[1]])
    #     zs[sk[coords[0], coords[1]]==n] = np.mean(zs[sk[coords]==n])

    # thresh = np.mean(zs)+np.std(zs)
    # zs = np.clip(zs, 0, np.mean(zs))
    # print(np.mean(zs), np.std(zs))
    # coords = coords[:,zs<thresh]
    # zs = zs[zs<thresh]

    # Transform coords to world coordinates
    coords = coords.astype(float)
    coords -= np.array([[cx], [cy]])
    coords[0] *= 1/fx
    coords[1] *= 1/fy
    coords = coords*zs
    coords = np.vstack([coords, zs, np.ones_like(zs)])
    points_c.append(coords.copy())
    coords = Rt[i] @ coords
    points.append(coords)
    frame_inds.extend([i for _ in range(coords.shape[1])])

points = np.hstack(points)
points = points[:3]

mins = [np.min(p[2]) for p in points_c]
# plt.figure()
# plt.hist(mins)
# plt.show()

In [181]:
# fig = plt.figure()
# ax = Axes3D(fig)
# ax.set_box_aspect([np.ptp(r) for r in np.hstack([points, pos_arr.T])])
# ax.set_xlabel('X')
# ax.set_zlabel('Z')
# ax.plot(*pos_arr.T)
# # ax.scatter(*old_pose_arr.T[1:4], c=colors)
# ax.scatter(*points)

# ax.quiver(*pos_arr.T, *(rot_mat_arr[:,:,0].T), color='blue', length=0.3)
# ax.quiver(*pos_arr.T, *(rot_mat_arr[:,:,1].T), color='red', length=0.1)
# ax.quiver(*pos_arr.T, *(rot_mat_arr[:,:,2].T), color='green', length=0.3)

In [56]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

coords = ['x', 'y', 'z']
fig = px.line_3d(pd.DataFrame({coords[i]:pos_arr[:,i] for i in range(3)}), x='x', y='y', z='z', labels='trajectory')
fig.add_trace(go.Scatter3d(
    x=pos_arr[:,0],
    y=pos_arr[:,1],
    z=pos_arr[:,2],
    marker={
        'color': np.arange(len(pos_arr)),
        # 'colorscale': 'viridis'
    },
    # hovertemplate='<b>%{text}</b>',
    # text=np.arange(len(pos_arr))
))
df = pd.DataFrame({
    'frame': frame_inds,
    'x': points[0],
    'y': points[1],
    'z': points[2],
})
# df = df[df.frame<5]

fig.add_trace(go.Scatter3d(
        x=df.x, y=df.y, z=df.z, mode='markers', name='skeleton points', marker={
            'size': 2,
            'color': df.frame,
            'colorscale': 'viridis'
        },
        hovertemplate='<b>%{text}</b>',
        text=df.frame
    ))

for q in range(3):
    fig.add_trace(go.Scatter3d(
        x=[Rt[0,0,3], Rt[0,0,3]+Rt[0,0,q]],
        y=[Rt[0,1,3], Rt[0,1,3]+Rt[0,1,q]],
        z=[Rt[0,2,3], Rt[0,2,3]+Rt[0,2,q]]
    ))

fig.write_html('figures/points.html', auto_open=True)